<h3>Week 3 Assignment on Segmenting and Clustering</h3>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import csv
import urllib
import re
%matplotlib inline

In [2]:
#Extract table from wiki
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

tabs = soup.find_all('table', class_='wikitable sortable')
#print(tabs)
#print(tabs.prettify())

type(tabs)

bs4.element.ResultSet

<h3> Extract Data from Wikipedia</h3>

In [3]:
tabs = tabs[0]
type(tabs)

bs4.element.Tag

In [6]:
#EXTRACT DATA AND WRITE TO CSV FILE
with open('wikidata.csv', 'a') as csvFile:
    for row in tabs.find_all('tr'):
        writer = csv.writer(csvFile)
        writer.writerow(row)

In [7]:
#Convert csv to dataframe
datadf = pd.read_csv('wikidata.csv')
#datadf = datadf.drop(['.1', '.2'], axis=1)

<h3>Transform Data to Dataframe</h3>

In [8]:
datadf.head(10)

,,<th>Postcode</th>,.1,<th>Borough</th>,.2,<th>Neighbourhood </th>
0,\n,<td>M1A</td>,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
1,\n,<td>M2A</td>,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
2,\n,<td>M3A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Parkwoods"" title=""Parkwoods..."
3,\n,<td>M4A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Victoria_Village"" title=""Vi..."
4,\n,<td>M5A</td>,\n,"<td><a href=""/wiki/Downtown_Toronto"" title=""Do...",\n,"<td><a href=""/wiki/Harbourfront_(Toronto)"" tit..."
5,\n,<td>M5A</td>,\n,"<td><a href=""/wiki/Downtown_Toronto"" title=""Do...",\n,"<td><a href=""/wiki/Regent_Park"" title=""Regent ..."
6,\n,<td>M6A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Lawrence_Heights"" title=""La..."
7,\n,<td>M6A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Lawrence_Manor"" title=""Lawr..."
8,\n,<td>M7A</td>,\n,"<td><a href=""/wiki/Queen%27s_Park_(Toronto)"" t...",\n,<td>Not assigned\n</td>
9,\n,<td>M8A</td>,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>


In [9]:
#Replace columns with column headers
datadf.columns = ["one","Postcode","two", "Borough","three", "Neighbourhood"]

In [10]:
#Strip unwanted characters out of Postcode
datadf['Postcode'] = datadf['Postcode'].map(lambda x: x.lstrip('<td>').rstrip('</td>'))
datadf.head()

,one,Postcode,two,Borough,three,Neighbourhood
0,\n,M1A,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
1,\n,M2A,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
2,\n,M3A,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Parkwoods"" title=""Parkwoods..."
3,\n,M4A,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Victoria_Village"" title=""Vi..."
4,\n,M5A,\n,"<td><a href=""/wiki/Downtown_Toronto"" title=""Do...",\n,"<td><a href=""/wiki/Harbourfront_(Toronto)"" tit..."


In [11]:
#Drop unwanted columns
datadf = datadf.drop(["one","two","three"], axis=1)

In [12]:
#"Clean column using Regular Expressions"
datadf['Borough'] = datadf['Borough'].str.extract(r'title="(.*?)">')
datadf.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,<td>Not assigned\n</td>
1,M2A,NaN,<td>Not assigned\n</td>
2,M3A,North York,"<td><a href=""/wiki/Parkwoods"" title=""Parkwoods..."
3,M4A,North York,"<td><a href=""/wiki/Victoria_Village"" title=""Vi..."
4,M5A,Downtown Toronto,"<td><a href=""/wiki/Harbourfront_(Toronto)"" tit..."


In [13]:
#"Clean column using Regular Expressions"
datadf['Neighbourhood'] = datadf['Neighbourhood'].str.extract(r'title="(.*?)">')
datadf.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park (Toronto),NaN
9,M8A,NaN,NaN


In [14]:
#Remove null data from Borough Column
datadf = datadf[pd.notnull(datadf['Borough'])]

In [15]:
datadf.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [16]:
datadf.groupby(['Postcode'])

In [17]:
#Replace Null Neighbourhood data with Borough
datadf.loc[(pd.isnull(datadf.Neighbourhood), 'Neighbourhood')] = datadf.Borough
datadf.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park (Toronto),Queen's Park (Toronto)
10,M9A,Etobicoke,Islington Avenue
11,M1B,"Scarborough, Toronto","Rouge, Toronto"
12,M1B,"Scarborough, Toronto","Malvern, Toronto"


In [18]:
datadf.shape

(208, 3)

<h3>Extracting Latitude and Longitude Data</h3>

In [19]:
#Convert csv to dataframe
latlongdf = pd.read_csv('Geospatial_Coordinates.csv')
latlongdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
latlongdf.columns = ["Postcode","Latitude","Longitude"]
latlongdf.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
#Merge data from Geospatial data, spot check for accuracy
df_merge_col = pd.merge(latlongdf, datadf, on='Postcode')
df_merge_col.head(10)

,Postcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,"Scarborough, Toronto","Rouge, Toronto"
1,M1B,43.806686,-79.194353,"Scarborough, Toronto","Malvern, Toronto"
2,M1C,43.784535,-79.160497,"Scarborough, Toronto",Highland Creek (Toronto)
3,M1C,43.784535,-79.160497,"Scarborough, Toronto",Rouge Hill
4,M1C,43.784535,-79.160497,"Scarborough, Toronto","Port Union, Toronto"
5,M1E,43.763573,-79.188711,"Scarborough, Toronto","Scarborough, Toronto"
6,M1E,43.763573,-79.188711,"Scarborough, Toronto","Morningside, Toronto"
7,M1E,43.763573,-79.188711,"Scarborough, Toronto","West Hill, Toronto"
8,M1G,43.770992,-79.216917,"Scarborough, Toronto","Woburn, Toronto"
9,M1H,43.773136,-79.239476,"Scarborough, Toronto","Scarborough, Toronto"


In [22]:
df_merge_col.shape

(208, 5)

<h2>Clustering Based on Neighborhood Data</h2>

In [23]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [24]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<h2>Create Map with Geolocators</h2>

In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge_col['Latitude'], df_merge_col['Longitude'], df_merge_col['Borough'], df_merge_col['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h2>Foursquare login</h2>

In [26]:
CLIENT_ID = 'SRGEUERUR2JNJJDKMQLM4VJNEZ131SYQN5PNB1JNWG4C0PTM' # your Foursquare ID
CLIENT_SECRET = 'VT2P2JUGLOSE03SSH4GLWTSXPYOFGMRGYE242PZCCPZSVUGM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SRGEUERUR2JNJJDKMQLM4VJNEZ131SYQN5PNB1JNWG4C0PTM
CLIENT_SECRET:VT2P2JUGLOSE03SSH4GLWTSXPYOFGMRGYE242PZCCPZSVUGM


In [27]:
df_merge_col.loc[0, 'Neighbourhood']

'Rouge, Toronto'

In [28]:
neighbourhood_latitude = df_merge_col.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_merge_col.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_merge_col.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rouge, Toronto are 43.806686299999996, -79.19435340000001.


In [29]:
tor_grouped = df_merge_col.groupby('Neighbourhood').mean().reset_index()
tor_grouped

,Neighbourhood,Latitude,Longitude
0,Agincourt North,43.815252,-79.284577
1,"Agincourt, Toronto",43.794200,-79.262029
2,"Alderwood, Toronto",43.602414,-79.543484
3,Bathurst Manor,43.754328,-79.442259
4,Bayview Village,43.786947,-79.385975
5,Beaumond Heights,43.739416,-79.588437
6,"Bedford Park, Toronto",43.733283,-79.419750
7,Berczy Park,43.644771,-79.373306
8,Birch Cliff,43.692657,-79.264848
9,CFB Toronto,43.737473,-79.464763


<h3>Create URL</h3>

In [36]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=SRGEUERUR2JNJJDKMQLM4VJNEZ131SYQN5PNB1JNWG4C0PTM&client_secret=VT2P2JUGLOSE03SSH4GLWTSXPYOFGMRGYE242PZCCPZSVUGM&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [37]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd848b3db04f559d8f4f5d3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h3>Get Nearby Venue</h3>

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
#Gather all venues neighourhoods
tor_venues = getNearbyVenues(names=df_merge_col['Neighbourhood'],
                                   latitudes=df_merge_col['Latitude'],
                                   longitudes=df_merge_col['Longitude']
                                  )


Rouge, Toronto
Malvern, Toronto
Highland Creek (Toronto)
Rouge Hill
Port Union, Toronto
Scarborough, Toronto
Morningside, Toronto
West Hill, Toronto
Woburn, Toronto
Scarborough, Toronto
Scarborough Village
Scarborough, Toronto
Ionview
Kennedy Park, Toronto
Clairlea
Golden Mile, Toronto
Oakridge, Toronto
Cliffcrest
Cliffside, Toronto
Scarborough, Toronto
Birch Cliff
Scarborough, Toronto
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale, Toronto
Wexford, Toronto
Agincourt, Toronto
Scarborough, Toronto
Scarborough, Toronto
Tam O'Shanter – Sullivan
Agincourt North
Scarborough, Toronto
Milliken, Ontario
Scarborough, Toronto
Scarborough, Toronto
Upper Rouge
Hillcrest Village
North York
Henry Farm
North York
Bayview Village
North York
York Mills
Newtonbrook
Willowdale, Toronto
North York
North York
Willowdale West
Parkwoods
North York
Flemingdon Park
North York
Bathurst Manor
North York
Wilson Heights, Toronto
Northwood Park
York University
CFB Toronto
North York
Downsview
North Yo

In [50]:
print(tor_venues.shape)
tor_venues.head()

(4397, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Toronto",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Toronto",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek (Toronto),43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Port Union, Toronto",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


In [52]:
tor_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt North,3,3,3,3,3,3
"Agincourt, Toronto",4,4,4,4,4,4
"Alderwood, Toronto",10,10,10,10,10,10
Bathurst Manor,19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
Beaumond Heights,9,9,9,9,9,9
"Bedford Park, Toronto",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57
Birch Cliff,4,4,4,4,4,4


In [53]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 273 uniques categories.


<h3>Initiate One Hot Encoding</h3>

In [54]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighbourhood'] = tor_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
manhattan_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Rouge, Toronto"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Malvern, Toronto"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Highland Creek (Toronto)
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Rouge Hill
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Port Union, Toronto"


In [55]:
tor_grouped = tor_onehot.groupby('Neighbourhood').mean().reset_index()
tor_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt North,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,"Agincourt, Toronto",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Alderwood, Toronto",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Bathurst Manor,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Beaumond Heights,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Bedford Park, Toronto",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Birch Cliff,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,CFB Toronto,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


<h3>Find The Top 5 Venues in Toronto for Each Neighbourhood</h3>

In [56]:
num_top_venues = 5

for hood in tor_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North----
                venue  freq
0          Playground  0.33
1         Coffee Shop  0.33
2                Park  0.33
3  Miscellaneous Shop  0.00
4       Movie Theater  0.00


----Agincourt, Toronto----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3        Skating Rink  0.25
4  Miscellaneous Shop  0.00


----Alderwood, Toronto----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2            Pool   0.1
3    Skating Rink   0.1
4  Sandwich Place   0.1


----Bathurst Manor----
                  venue  freq
0           Coffee Shop  0.11
1  Fast Food Restaurant  0.05
2         Shopping Mall  0.05
3        Sandwich Place  0.05
4           Supermarket  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4        Moving Target  0.00


----Beaumond Heights----
      

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Park,Playground,Coffee Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,"Agincourt, Toronto",Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
2,"Alderwood, Toronto",Pizza Place,Pharmacy,Skating Rink,Athletics & Sports,Sandwich Place,Pub,Pool,Gym,Coffee Shop,Donut Shop
3,Bathurst Manor,Coffee Shop,Pet Store,Sandwich Place,Restaurant,Supermarket,Deli / Bodega,Bank,Fried Chicken Joint,Frozen Yogurt Shop,Fast Food Restaurant
4,Bayview Village,Bank,Café,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


<h3>Initiate Clustering for Each Neighbourhood</h3>

In [59]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [81]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df_merge_col

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tor_merged.head() # check the last columns!

,Postcode,Latitude,Longitude,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.806686,-79.194353,"Scarborough, Toronto","Rouge, Toronto",4.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant
1,M1B,43.806686,-79.194353,"Scarborough, Toronto","Malvern, Toronto",4.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant
2,M1C,43.784535,-79.160497,"Scarborough, Toronto",Highland Creek (Toronto),2.0,Bar,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,M1C,43.784535,-79.160497,"Scarborough, Toronto",Rouge Hill,2.0,Bar,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
4,M1C,43.784535,-79.160497,"Scarborough, Toronto","Port Union, Toronto",2.0,Bar,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [72]:
tor_merged.dtypes

Postcode                   object
Latitude                  float64
Longitude                 float64
Borough                    object
Neighbourhood              object
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster Labels' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters